##### Testing output from an api run


In [4]:
import pydantic
import pandas as pd
import os
from data_models import Entity, Labels
import rich

In [3]:
obj = Labels(
    Entities=[
        Entity(text='Systems Engineering Manager', type='JOB_TITLE'),
        Entity(text='Technical Manager', type='SOFT_SKILL'),
        Entity(text='experience', type='PERSONALITY_TRAIT'),
        Entity(text='knowledge', type='PERSONALITY_TRAIT'),
        Entity(text='hardware engineering', type='CAPABILITY'),
        Entity(text='software engineering', type='CAPABILITY'),
        Entity(text='subcontracts management', type='CAPABILITY'),
        Entity(text='Technical Management', type='SOFT_SKILL'),
        Entity(text='MS Project', type='CAPABILITY'),
        Entity(text='Agile Software Development', type='CAPABILITY'),
        Entity(text='Jira', type='CAPABILITY'),
        Entity(text='Handsoft', type='CAPABILITY'),
        Entity(text='ePDM', type='CAPABILITY'),
        Entity(text='Data Analysis', type='CAPABILITY'),
        Entity(text='Customer Management Systems', type='CAPABILITY'),
        Entity(text='Hardware-Software Integration', type='CAPABILITY'),
        Entity(text='Rational Requisite Pro', type='CAPABILITY'),
        Entity(text='ClearQuest', type='CAPABILITY'),
        Entity(text='Visio', type='CAPABILITY'),
        Entity(text='DOORS', type='CAPABILITY'),
        Entity(text='LEAN/Six Sigma', type='CAPABILITY'),
        Entity(text='Safety Engineering', type='CAPABILITY'),
        Entity(text='Requirements Engineering Creation', type='CAPABILITY'),
        Entity(text='System Architecture Design', type='CAPABILITY'),
        Entity(text='Creo', type='CAPABILITY'),
        Entity(text='Solidworks', type='CAPABILITY'),
        Entity(text='ANSYS 14', type='CAPABILITY'),
        Entity(text='DFMA', type='CAPABILITY'),
        Entity(text='DFA/DFM', type='CAPABILITY'),
        Entity(text='AutoCAD', type='CAPABILITY'),
        Entity(text='Zemax', type='CAPABILITY'),
        Entity(text='Unigraphics', type='CAPABILITY'),
        Entity(text='I-deas', type='CAPABILITY'),
        Entity(text='Intralink', type='CAPABILITY'),
        Entity(text='Windchill', type='CAPABILITY'),
        Entity(text='Drafting Engineering Drawings', type='CAPABILITY'),
        Entity(text='GD&T', type='CAPABILITY'),
        Entity(text='Tolerances', type='CAPABILITY'),
        Entity(text='Bill of Materials', type='CAPABILITY'),
        Entity(text='Cage Codes', type='CAPABILITY'),
        Entity(text='ASME Standards', type='CAPABILITY'),
        Entity(text='Weld and Material Callouts', type='CAPABILITY'),
        Entity(text='Development', type='CAPABILITY'),
        Entity(text='C++', type='CAPABILITY'),
        Entity(text='JAVA', type='CAPABILITY'),
        Entity(text='MathCAD', type='CAPABILITY'),
        Entity(text='Matlab', type='CAPABILITY'),
        Entity(text='Simulink', type='CAPABILITY'),
        Entity(text='Microsoft Visual Studio', type='CAPABILITY'),
        Entity(text='Active Secret Security Clearance', type='CAPABILITY'),
        Entity(text='Technical Program Manager', type='JOB_TITLE'),
        Entity(text='LCS Game Based Learning', type='CAPABILITY'),
        Entity(text='Systems Engineers', type='CAPABILITY'),
        Entity(text='Software Engineers', type='CAPABILITY'),
        Entity(text='Test Engineers', type='CAPABILITY'),
        Entity(text='Technical Artists', type='CAPABILITY'),
        Entity(text='Game Developers', type='CAPABILITY'),
        Entity(text='budget', type='PERSONALITY_TRAIT'),
        Entity(text='scope', type='PERSONALITY_TRAIT'),
        Entity(text='schedule', type='PERSONALITY_TRAIT'),
        Entity(text='cost savings', type='PERSONALITY_TRAIT'),
        Entity(text='enterprise architecture', type='CAPABILITY'),
        Entity(text='tool suite for game development', type='CAPABILITY'),
        Entity(text='product design', type='CAPABILITY'),
        Entity(text='schedule and budget reviews', type='CAPABILITY'),
        Entity(text='subcontractors', type='CAPABILITY'),
        Entity(text='Mobile based training platform', type='CAPABILITY'),
        Entity(text='trainees', type='PERSONALITY_TRAIT'),
        Entity(text='design of over $20M worth of heavy equipment', type='CAPABILITY'),
        Entity(text='multimillion-dollar simulated Launch and Recovery Systems', type='CAPABILITY'),
        Entity(text='proposal-pricing and estimating-efforts', type='CAPABILITY'),
        Entity(text='training courseware', type='CAPABILITY'),
        Entity(text='Learning Objective environment architecture', type='CAPABILITY'),
        Entity(text='sample product demonstrating level 3 IMI training', type='CAPABILITY'),
        Entity(text='pricing model and technical manpower estimates', type='CAPABILITY'),
        Entity(text='M134a Dillon Minigun Simulator', type='CAPABILITY'),
        Entity(text='System Design and System Requirements', type='CAPABILITY'),
        Entity(text='large projection screens', type='CAPABILITY'),
        Entity(text='simulated weapon and computing hardware', type='CAPABILITY'),
        Entity(text='system simulating the timing of the feeder/delinker', type='CAPABILITY'),
        Entity(text='Bradley Fighting Vehicle Simulator', type='CAPABILITY'),
        Entity(text='performance award', type='PERSONALITY_TRAIT'),
        Entity(text='opto-mechanical design', type='CAPABILITY'),
        Entity(text='HVAC system', type='CAPABILITY'),
        Entity(text='heat sinks', type='CAPABILITY'),
        Entity(text='optical and visual instruments', type='CAPABILITY'),
        Entity(text='stress and strain analysis', type='CAPABILITY'),
        Entity(text='material selection', type='CAPABILITY'),
        Entity(text='vibration and shock analysis', type='CAPABILITY'),
        Entity(text='manufacturability analysis', type='CAPABILITY'),
        Entity(text='RTV bonding', type='CAPABILITY'),
        Entity(text='elastomeric materials', type='CAPABILITY'),
        Entity(text='polyurethane foam', type='CAPABILITY'),
        Entity(text='vibration and shock support', type='CAPABILITY'),
        Entity(text='mounting system', type='CAPABILITY'),
        Entity(text='OLED screen and circuit board', type='CAPABILITY'),
        Entity(text='FARO arm', type='CAPABILITY'),
        Entity(text='damage assessment on a fighter aircraft (MIG-29A)', type='CAPABILITY'),
        Entity(text='fragmentation and blast damage', type='CAPABILITY'),
        Entity(text='physics system', type='CAPABILITY'),
        Entity(text='six degree of freedom physics model', type='CAPABILITY'),
        Entity(text='water flow and pressure', type='CAPABILITY'),
        Entity(text='shearing of female reproductive muscles', type='CAPABILITY'),
        Entity(text='watercraft accidents', type='CAPABILITY'),
        Entity(text='stress on the muscle', type='CAPABILITY'),
        Entity(text='Bilingual Arabic/English', type='PERSONALITY_TRAIT'),
        Entity(text='US Citizen', type='PERSONALITY_TRAIT'),
        Entity(text='Bachelor of Science', type='PERSONALITY_TRAIT'),
        Entity(text='Mechanical Engineering', type='CAPABILITY'),
        Entity(text='Enterprise System Architecture', type='CAPABILITY')
    ]
)

In [6]:
obj_dict = obj.model_dump()
rich.print(obj_dict)

{
    'Entities': [
        {'text': 'Systems Engineering Manager', 'type': 'JOB_TITLE'},
        {'text': 'Technical Manager', 'type': 'SOFT_SKILL'},
        {'text': 'experience', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'knowledge', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'hardware engineering', 'type': 'CAPABILITY'},
        {'text': 'software engineering', 'type': 'CAPABILITY'},
        {'text': 'subcontracts management', 'type': 'CAPABILITY'},
        {'text': 'Technical Management', 'type': 'SOFT_SKILL'},
        {'text': 'MS Project', 'type': 'CAPABILITY'},
        {'text': 'Agile Software Development', 'type': 'CAPABILITY'},
        {'text': 'Jira', 'type': 'CAPABILITY'},
        {'text': 'Handsoft', 'type': 'CAPABILITY'},
        {'text': 'ePDM', 'type': 'CAPABILITY'},
        {'text': 'Data Analysis', 'type': 'CAPABILITY'},
        {'text': 'Customer Management Systems', 'type': 'CAPABILITY'},
        {'text': 'Hardware-Software Integration', 'type': 'CAPABILITY'},
        {'text': 'Rational Requisite Pro', 'type': 'CAPABILITY'},
        {'text': 'ClearQuest', 'type': 'CAPABILITY'},
        {'text': 'Visio', 'type': 'CAPABILITY'},
        {'text': 'DOORS', 'type': 'CAPABILITY'},
        {'text': 'LEAN/Six Sigma', 'type': 'CAPABILITY'},
        {'text': 'Safety Engineering', 'type': 'CAPABILITY'},
        {'text': 'Requirements Engineering Creation', 'type': 'CAPABILITY'},
        {'text': 'System Architecture Design', 'type': 'CAPABILITY'},
        {'text': 'Creo', 'type': 'CAPABILITY'},
        {'text': 'Solidworks', 'type': 'CAPABILITY'},
        {'text': 'ANSYS 14', 'type': 'CAPABILITY'},
        {'text': 'DFMA', 'type': 'CAPABILITY'},
        {'text': 'DFA/DFM', 'type': 'CAPABILITY'},
        {'text': 'AutoCAD', 'type': 'CAPABILITY'},
        {'text': 'Zemax', 'type': 'CAPABILITY'},
        {'text': 'Unigraphics', 'type': 'CAPABILITY'},
        {'text': 'I-deas', 'type': 'CAPABILITY'},
        {'text': 'Intralink', 'type': 'CAPABILITY'},
        {'text': 'Windchill', 'type': 'CAPABILITY'},
        {'text': 'Drafting Engineering Drawings', 'type': 'CAPABILITY'},
        {'text': 'GD&T', 'type': 'CAPABILITY'},
        {'text': 'Tolerances', 'type': 'CAPABILITY'},
        {'text': 'Bill of Materials', 'type': 'CAPABILITY'},
        {'text': 'Cage Codes', 'type': 'CAPABILITY'},
        {'text': 'ASME Standards', 'type': 'CAPABILITY'},
        {'text': 'Weld and Material Callouts', 'type': 'CAPABILITY'},
        {'text': 'Development', 'type': 'CAPABILITY'},
        {'text': 'C++', 'type': 'CAPABILITY'},
        {'text': 'JAVA', 'type': 'CAPABILITY'},
        {'text': 'MathCAD', 'type': 'CAPABILITY'},
        {'text': 'Matlab', 'type': 'CAPABILITY'},
        {'text': 'Simulink', 'type': 'CAPABILITY'},
        {'text': 'Microsoft Visual Studio', 'type': 'CAPABILITY'},
        {'text': 'Active Secret Security Clearance', 'type': 'CAPABILITY'},
        {'text': 'Technical Program Manager', 'type': 'JOB_TITLE'},
        {'text': 'LCS Game Based Learning', 'type': 'CAPABILITY'},
        {'text': 'Systems Engineers', 'type': 'CAPABILITY'},
        {'text': 'Software Engineers', 'type': 'CAPABILITY'},
        {'text': 'Test Engineers', 'type': 'CAPABILITY'},
        {'text': 'Technical Artists', 'type': 'CAPABILITY'},
        {'text': 'Game Developers', 'type': 'CAPABILITY'},
        {'text': 'budget', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'scope', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'schedule', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'cost savings', 'type': 'PERSONALITY_TRAIT'},
        {'text': 'enterprise architecture', 'type': 'CAPABILITY'},
        {'text': 'tool suite for game development', 'type': 'CAPABILITY'},
        {'text': 'product design', 'type': 'CAPABILITY'},
        {'text': 'schedule and budget reviews', 'type': 'CAPABILITY'},
        {'text': 'subcontractors', 'type': 'CAPABILITY'},
        {'text': 'Mobile based training platform', 'type': 'CAPABILITY'},
        {

In [ ]:
df = pd.DataFrame(obj_dict['Entities'])
contents = df['text'].values.tolist()
labels = df['type'].values.tolist()


Systems Engineering Manager


In [ ]:
def validate_entities(contents:list,source_filename:str):
    with open(f'{os.getcwd()}/data_src/{source_filename}.txt','r') as f:
        doc = f.read()

    for text in contents:
        if text not in doc:
            continue
        else:
            